# Train using Word Embedding, Dense Layer, and Keras
Reference: https://www.kaggle.com/nzw0301/simple-keras-fasttext-val-loss-0-31/notebook  

In [23]:
import pandas as pd
import numpy as np
from multiprocessing import cpu_count

import keras

import spacy

!python -m spacy download en_core_web_sm
!python -m spacy validate

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.4) ======================
ℹ spaCy installation:
/home/ec2-user/anaconda3/lib/python3.7/site-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.1.0   ✔



# Read Data Set

In [2]:
filename_dataset = './datasets/train.csv'
df = pd.read_csv(filename_dataset)

df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [3]:
y = df['author'].map({'EAP':0, 'HPL':1, 'MWS':2})
y.head()

0    0
1    1
2    0
3    2
4    1
Name: author, dtype: int64

In [4]:
y = keras.utils.to_categorical(y)

In [5]:
y.shape

(19579, 3)

In [6]:
y[:5]

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

# Preprocessing
### Low-frequency words
In my experience, fastText is very fast, but I need to delete rare words to avoid overfitting.

##### NOTE: Some keywords are rare words, such like Cthulhu in Cthulhu Mythos of Howard Phillips Lovecraft. But these are useful for this task.

### Removing Stopwords
Nothing. To identify author from a sentence, some stopwords play an important role because one has specific usages of them.

### Stemming and Lowercase
Nothing. This reason is the same for stopwords removing. And I guess some stemming rules provided by libraries is bad for this task because all author is the older author.

### Cutting long sentence
Too long documents are cut.

### Punctuation
Because I guess each author has unique punctuations's usage in the novel, I separate them from words.

e.g. Don't worry -> Don ' t worry

In [7]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'tagger'])
nlp.pipeline

[]

# Define Function to Insert White Spaces around Punctuations

In [8]:
def brief_cleaning(raw_docs):
    """
    Normalize the docs and insert white spaces around each puncutation
    """
    punctuations = set(',.:;\"?!')
    
    for doc in raw_docs:
        doc = doc.lower()
        doc = doc.replace('\'', ' \' ')
        
        intersections = set(doc).intersection(punctuations)
        
        if not intersections:
            yield doc
        else:
            # insert white spaces around punctuations
            for punct in intersections:
                doc = doc.replace(punct, ' {} '.format(punct))
                
            yield doc
            

def preprocess(doc_spacy):
    """
    Lemmatize
    
    Args:
        doc_spacy (spacy.tokens.doc.Doc):
    """
    
    return [token.lemma_ for token in doc_spacy if not token.is_space]

In [9]:
text = df['text'][0]
print(text)

This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.


In [10]:
text.lower().replace('\'', ' \' ')

punctuations = set(',.:;\"?!')
intersections = set(text).intersection(punctuations)

print(intersections)

if not intersections:
    print('do nothing')
else:
    for punct in intersections:
        text = text.replace(punct, ' {} '.format(punct))
    
print(text)

{',', ';', '.'}
This process ,  however ,  afforded me no means of ascertaining the dimensions of my dungeon ;  as I might make its circuit ,  and return to the point whence I set out ,  without being aware of the fact ;  so perfectly uniform seemed the wall . 


In [11]:
print([token.lemma_ for token in nlp(text) if not token.is_space])

['This', 'process', ',', 'however', ',', 'afford', 'me', 'no', 'mean', 'of', 'ascertain', 'the', 'dimension', 'of', 'my', 'dungeon', ';', 'a', 'I', 'may', 'make', 'its', 'circuit', ',', 'and', 'return', 'to', 'the', 'point', 'whence', 'I', 'set', 'out', ',', 'without', 'be', 'aware', 'of', 'the', 'fact', ';', 'so', 'perfectly', 'uniform', 'seem', 'the', 'wall', '.']


# Start Preprocessing

In [12]:
%time docs = [preprocess(doc) for doc in nlp.pipe(brief_cleaning(df['text']), batch_size=100, n_threads=cpu_count())]

CPU times: user 8.53 s, sys: 33 ms, total: 8.56 s
Wall time: 8.6 s


In [13]:
len(docs)

19579

In [14]:
print(docs[0])

['this', 'process', ',', 'however', ',', 'afford', 'me', 'no', 'mean', 'of', 'ascertain', 'the', 'dimension', 'of', 'my', 'dungeon', ';', 'a', 'i', 'may', 'make', 'its', 'circuit', ',', 'and', 'return', 'to', 'the', 'point', 'whence', 'i', 'set', 'out', ',', 'without', 'be', 'aware', 'of', 'the', 'fact', ';', 'so', 'perfectly', 'uniform', 'seem', 'the', 'wall', '.']


In [24]:
np.max(docs)

['\ufeff1', 'of', 'all', 'i', 'dismember', 'the', 'corpse', '.']

# Create Dictionary

In [15]:
tokenizer = keras.preprocessing.text.Tokenizer(filters='')

In [16]:
# tokenizer.fit_on_texts([token for doc in docs[:2] for token in doc])
tokenizer.fit_on_texts(docs)

In [22]:
len(tokenizer.word_counts)

18048

In [17]:
print(docs[0])

['this', 'process', ',', 'however', ',', 'afford', 'me', 'no', 'mean', 'of', 'ascertain', 'the', 'dimension', 'of', 'my', 'dungeon', ';', 'a', 'i', 'may', 'make', 'its', 'circuit', ',', 'and', 'return', 'to', 'the', 'point', 'whence', 'i', 'set', 'out', ',', 'without', 'be', 'aware', 'of', 'the', 'fact', ';', 'so', 'perfectly', 'uniform', 'seem', 'the', 'wall', '.']


In [18]:
print(tokenizer.texts_to_sequences(docs[0:2]))

[[23, 2200, 1, 151, 1, 646, 27, 38, 170, 3, 1508, 2, 1851, 3, 13, 3312, 14, 7, 9, 58, 64, 49, 3699, 1, 5, 175, 8, 2, 188, 1930, 9, 299, 86, 1, 145, 6, 1009, 3, 2, 303, 14, 40, 1509, 4228, 94, 2, 232, 4], [16, 107, 134, 592, 8, 27, 12, 2, 3700, 58, 6, 7, 449, 889, 4]]


In [19]:
tokenizer.index_word[23]

'this'

In [20]:
tokenizer.index_word[2200]

'process'

# Encode the Train Dataset

In [56]:
docs_encoded = tokenizer.texts_to_sequences(docs)

In [57]:
type(docs_encoded)

list

In [58]:
len(docs_encoded)

19579

In [59]:
seq_length = 256
docs_encoded = keras.preprocessing.sequence.pad_sequences(sequences=docs_encoded, 
                                                          maxlen=seq_length,
                                                          dtype='int32',
                                                         padding='post',
                                                         value=0)

In [60]:
type(docs_encoded)

numpy.ndarray

In [61]:
docs_encoded.shape

(19579, 256)

In [64]:
docs_encoded[:2, :30]

array([[  23, 2200,    1,  151,    1,  646,   27,   38,  170,    3, 1508,
           2, 1851,    3,   13, 3312,   14,    7,    9,   58,   64,   49,
        3699,    1,    5,  175,    8,    2,  188, 1930],
       [  16,  107,  134,  592,    8,   27,   12,    2, 3700,   58,    6,
           7,  449,  889,    4,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

# Define DNN Model

In [74]:
def create_model(input_dim, vocab_size, embedding_size=128, output_dim=3, lr=1e-3):
    
    # Input shape does not include batch_size: (None, seq_length)
    inputs = keras.layers.Input(input_dim[1:])
    
    x_embed = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_size)(inputs)
    
    x_avg = keras.layers.GlobalAveragePooling1D()(x_embed)
    
    x_fc1 = keras.layers.Dense(output_dim)(x_avg)
    
    outputs = keras.layers.Activation('softmax')(x_fc1)
    
    model = keras.models.Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(lr=lr)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])
    return model


# Vocab size includes zero-padding
vocab_size = len(tokenizer.word_counts) + 1 
input_dim = docs_encoded.shape

model = create_model(input_dim=input_dim, vocab_size=vocab_size, embedding_size=128)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 256, 128)          2310272   
_________________________________________________________________
global_average_pooling1d_5 ( (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_5 (Activation)    (None, 3)                 0         
Total params: 2,310,659
Trainable params: 2,310,659
Non-trainable params: 0
_________________________________________________________________


In [75]:
epochs=100
%time model.fit(docs_encoded, y, batch_size=128, epochs=epochs, callbacks=[keras.callbacks.EarlyStopping(patience=2, monitor='loss')])

Epoch 1/100
19579/19579 [==============================] - 5s 268us/step - loss: 1.0798 - acc: 0.4039
Epoch 2/100
19579/19579 [==============================] - 5s 253us/step - loss: 1.0384 - acc: 0.4351
Epoch 3/100
19579/19579 [==============================] - 5s 250us/step - loss: 0.9493 - acc: 0.5914
Epoch 4/100
19579/19579 [==============================] - 5s 246us/step - loss: 0.8369 - acc: 0.7141
Epoch 5/100
19579/19579 [==============================] - 5s 248us/step - loss: 0.7368 - acc: 0.7593
Epoch 6/100
19579/19579 [==============================] - 5s 246us/step - loss: 0.6586 - acc: 0.7852
Epoch 7/100
19579/19579 [==============================] - 5s 258us/step - loss: 0.5961 - acc: 0.8058
Epoch 8/100
19579/19579 [==============================] - 5s 258us/step - loss: 0.5463 - acc: 0.8191
Epoch 9/100
19579/19579 [==============================] - 5s 254us/step - loss: 0.5053 - acc: 0.8354
Epoch 10/100
19579/19579 [==============================] - 5s 257us/step - loss: 

19579/19579 [==============================] - 5s 254us/step - loss: 0.0505 - acc: 0.9859
Epoch 81/100
19579/19579 [==============================] - 5s 242us/step - loss: 0.0480 - acc: 0.9876
Epoch 82/100
19579/19579 [==============================] - 5s 241us/step - loss: 0.0474 - acc: 0.9884
Epoch 83/100
19579/19579 [==============================] - 5s 250us/step - loss: 0.0457 - acc: 0.9878
Epoch 84/100
19579/19579 [==============================] - 5s 251us/step - loss: 0.0443 - acc: 0.9888
Epoch 85/100
19579/19579 [==============================] - 5s 252us/step - loss: 0.0433 - acc: 0.9892
Epoch 86/100
19579/19579 [==============================] - 5s 252us/step - loss: 0.0428 - acc: 0.9895
Epoch 87/100
19579/19579 [==============================] - 5s 247us/step - loss: 0.0420 - acc: 0.9897
Epoch 88/100
19579/19579 [==============================] - 5s 248us/step - loss: 0.0409 - acc: 0.9895
Epoch 89/100
19579/19579 [==============================] - 5s 249us/step - loss: 0.04